In [1]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re

with open('../bin/textbook-urls.txt', 'r') as f:
    urls = f.read().splitlines()

## requests_html

Problems..

In [2]:
import nest_asyncio
nest_asyncio.apply()
from requests_html import AsyncHTMLSession
asession = AsyncHTMLSession()

In [8]:
async def getFirstPage():
    r = await asession.get(urls[0])
    await r.html.arender()
    return r

res = asession.run(getFirstPage)

## Selenium

In [23]:
from webdriver_manager.chrome import ChromeDriverManager
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/chromium-browser is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x558dd4ab6463 <unknown>
#1 0x558dd487a8d8 <unknown>
#2 0x558dd48a2b6a <unknown>
#3 0x558dd489dc05 <unknown>
#4 0x558dd48e1802 <unknown>
#5 0x558dd48e12af <unknown>
#6 0x558dd48d9443 <unknown>
#7 0x558dd48aa3c5 <unknown>
#8 0x558dd48ab531 <unknown>
#9 0x558dd4b08dce <unknown>
#10 0x558dd4b0c192 <unknown>
#11 0x558dd4aed93e <unknown>
#12 0x558dd4b0d103 <unknown>
#13 0x558dd4ae0d85 <unknown>
#14 0x558dd4b2e0a8 <unknown>
#15 0x558dd4b2e239 <unknown>
#16 0x558dd4b49492 <unknown>
#17 0x7f6c5757d609 start_thread


In [15]:
ChromeDriverManager().install()

'/home/jovyan/.wdm/drivers/chromedriver/linux64/107.0.5304/chromedriver'

In [11]:
!echo 'export PATH=$PATH:/home/jovyan/.wdm/drivers/chromedriver/linux64/107.0.5304/chromedriver' >> ~/.bash_profile
!source ~/.bash_profile

In [14]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

service = Service(executable_path='/home/jovyan/.wdm/drivers/chromedriver/linux64/107.0.5304/chromedriver')
driver = webdriver.Chrome(service=service)

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x555a1b892463 <unknown>
#1 0x555a1b6568d8 <unknown>
#2 0x555a1b67c04a <unknown>
#3 0x555a1b679c05 <unknown>
#4 0x555a1b6bd802 <unknown>
#5 0x555a1b6bd2af <unknown>
#6 0x555a1b6b5443 <unknown>
#7 0x555a1b6863c5 <unknown>
#8 0x555a1b687531 <unknown>
#9 0x555a1b8e4dce <unknown>
#10 0x555a1b8e8192 <unknown>
#11 0x555a1b8c993e <unknown>
#12 0x555a1b8e9103 <unknown>
#13 0x555a1b8bcd85 <unknown>
#14 0x555a1b90a0a8 <unknown>
#15 0x555a1b90a239 <unknown>
#16 0x555a1b925492 <unknown>
#17 0x7fb5ea326609 start_thread


In [12]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.get(my_url)
p_element = driver.find_element_by_id(id_='intro-text')
print(p_element.text)
# result:
'Yay! Supports javascript'

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [48]:
def getText(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'html.parser')
    all_text = soup.get_text().lstrip('Learning Objectives').rstrip('PreviousNext')
    selection_text = re.sub('PreviousNext.+', '', all_text, flags=re.DOTALL)
    selection_text = re.sub('.+Learning Objectives', '', selection_text, flags=re.DOTALL)
    return(selection_text)
urls[:10]

['https://openstax.org/details/books/algebra-and-trigonometry/pages/preface',
 'https://openstax.org/details/books/algebra-and-trigonometry-2e/pages/preface',
 'https://openstax.org/details/books/calculus-volume-1/pages/preface',
 'https://openstax.org/details/books/calculus-volume-2/pages/preface',
 'https://openstax.org/details/books/calculus-volume-3/pages/preface',
 'https://openstax.org/details/books/college-algebra/pages/preface',
 'https://openstax.org/details/books/college-algebra-2e/pages/preface',
 'https://openstax.org/details/books/college-algebra-corequisite-support/pages/preface',
 'https://openstax.org/details/books/college-algebra-corequisite-support-2e/pages/preface',
 'https://openstax.org/details/books/elementary-algebra-2e/pages/preface']

In [64]:
def ScrapeTextbook(URL):
    textbook_split = URL.split('/')
    textbook_suffix = textbook_split[-1]
    textbook_prefix = '/'.join(textbook_split[:-1])+'/'
    print(textbook_prefix+textbook_suffix)
    
    textbook = "https://openstax.org/books/us-history/pages/"
    #req = Request(textbook + "1-introduction")
    req = Request(textbook_suffix + textbook_prefix)
    html_page = urlopen(req)
    soup = BeautifulSoup(html_page, "html.parser")

    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))
    
    section_urls = []
    for link in links:
        match = re.search('^[1234567890]+-', link)
        if match:
            if link.split('-')[1].isnumeric():
                section_urls.append(textbook + link)
                
    section_dict = {'section':[], 'text':[]}
    for section_url in section_urls:
        section_number = '-'.join(section_url.split('/')[-1].split('-')[:2])
        section_dict['section'].append(section_number)
        section_dict['text'].append(getText(section_url))
    
    return section_dict

In [67]:
ScrapeTextbook("https://openstax.org/books/us-history/pages/1-introduction")

https://openstax.org/books/us-history/pages/1-introduction


URLError: <urlopen error unknown url type: 1-introductionhttps>